<H1>Algoritmo Genético</H1>
<h2>N-Queens Problam</h2>
<p><b>Objeito: </b>Posicionar as rainhas no tabuleiro de forma que elas não se ataquem</p>


In [1]:
# Importar as bibliotecas
import math
import random
import copy
import itertools as it
TAMANHOTABULEIRODEFATUL = 8

In [2]:
# Gerar tabuleiro com matriz t por t de zeros com 1 (rainhas) aletóriamente
def geraTabuleiro(tamanho):
    if tamanho<2: return[-1]
    tab = []
    for i in range(tamanho):
        l=[]
        for j in range(tamanho):
            l.append(0)
        tab.append(l)
    for i in range(tamanho): 
        while True:
            j = random.randint(0,tamanho-1)
            if tab[i][j] == 0:
                tab[i][j]=1
                break
    return tab

In [3]:
# Testa geraTabuleiro()
geraTabuleiro(8)

[[0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0]]

In [4]:
# Localiza rainhas
def locateQueens(estado):
    q=[]
    for i in range(len(estado)):
        for j in range(len(estado)):
            if estado[i][j] == 1: q.append((i,j))
    return q

In [5]:
print(locateQueens(geraTabuleiro(8)))

[(0, 4), (1, 6), (2, 0), (3, 0), (4, 3), (5, 7), (6, 0), (7, 1)]


In [6]:
# Função para calcular os ataques possiveis no tabuleiro
def calAtaques(estado):
    at=0
    atlinhas=0
    # ataque em linhas
    for i in estado:
        if sum(i)>1:
            atlinhas+=sum(i)-1
    at+=atlinhas
    
    # ataques em colunas
    atcolunas = 0
    for c in range(len(estado)):
        n1 = 0
        for l in range(len(estado)): n1 += estado[l][c]
        if n1>1:
            atcolunas += n1 - 1
    at+=atcolunas
    pr = locateQueens(estado)
    d=0
    # Ataques em diagonais
    for p, coords in enumerate(pr):
        i,j = coords
        # diagonal esquerda
        k,w = i+1,j-1
        while(w>=0 and k<len(estado)):
            if estado[k][w]==1:
                d+=1
                break
            w-=1
            k+=1
        # diagonal direita
        l,c = i+1, j+1
        while(1 < len(estado) and c<len(estado)):
            if estado[1][c]==1:
                d+=1
                break
            l+=1
            c+=1
    at+=d
    return at

In [7]:
print(calAtaques(geraTabuleiro(8)))

7


In [8]:
# Função de Mutação
def mutacao(estado, passo):
    tam = len(estado)
    while True:
        i,j = random.choice(locateQueens(estado))
        k = (j+passo)%tam
        if estado[i][k] == 0:
            estado[i][k],estado[i][j] = estado[i][j],estado[i][k]
            break
    return estado

In [9]:
tab = geraTabuleiro(8)
print("Ataques:",calAtaques(tab))
print("Posição:",locateQueens(tab))
tab = mutacao(tab,2)
print("Ataques:",calAtaques(tab))
print("Posição:",locateQueens(tab))

Ataques: 6
Posição: [(0, 6), (1, 3), (2, 2), (3, 0), (4, 3), (5, 4), (6, 1), (7, 7)]
Ataques: 6
Posição: [(0, 6), (1, 3), (2, 4), (3, 0), (4, 3), (5, 4), (6, 1), (7, 7)]


In [10]:
# Gerar população inicial (N individuos de tamanho tam)
def geraPopulacaoInicial(N,tam): 
    populacao = []
    for i in range(N):
        populacao.append(geraTabuleiro(tam))
    return populacao

In [62]:
# Testar função geraPopulacaoInicial()
pop = geraPopulacaoInicial(100, 8)

In [12]:
print(pop)

[[[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]], [[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], [[0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0]], [[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0]], [[0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]]


In [13]:
# Função de cruzamento
def crossOver(genoma1, genoma2):
    g1 = copy.deepcopy(genoma1)
    g2 = copy.deepcopy(genoma2)
    corte = random.randint(0,len(g1)-1)
    return g1[:corte]+g2[corte:], g2[:corte]+g1[corte:] # dois filhos gerados

In [14]:
crossOver(geraTabuleiro(4),geraTabuleiro(4))

([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]],
 [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0]])

In [56]:
# Função fitness - Retorna o indice fitness da população toda
def fitness(populacao):
    fits=[]
    tam = len(populacao[0])
    maxAtaques = math.factorial(tam)/(2*math.factorial(tam-2))
    for i in populacao:
        fits.append(1- calAtaques(i)/maxAtaques)
    return fits

In [81]:
# Testar Função fitness
fit = fitness(pop)
print(fit)

[0.75, 0.6428571428571428, 0.75, 0.8571428571428572, 0.7857142857142857, 0.6428571428571428, 0.5357142857142857, 0.7142857142857143, 0.6428571428571428, 0.6428571428571428, 0.8214285714285714, 0.5357142857142857, 0.6071428571428572, 0.6785714285714286, 0.6071428571428572, 0.7857142857142857, 0.75, 0.6428571428571428, 0.6785714285714286, 0.6071428571428572, 0.7857142857142857, 0.7142857142857143, 0.75, 0.6785714285714286, 0.5714285714285714, 0.6785714285714286, 0.8571428571428572, 0.6428571428571428, 0.8571428571428572, 0.75, 0.6428571428571428, 0.5714285714285714, 0.7857142857142857, 0.75, 0.8571428571428572, 0.8928571428571429, 0.8928571428571429, 0.6785714285714286, 0.6071428571428572, 0.7142857142857143, 0.5714285714285714, 0.75, 0.7857142857142857, 0.6785714285714286, 0.6428571428571428, 0.6071428571428572, 0.6428571428571428, 0.6428571428571428, 0.8214285714285714, 0.75, 0.8928571428571429, 0.6785714285714286, 0.75, 0.7857142857142857, 0.6428571428571428, 0.9285714285714286, 0.821

In [17]:
# Função da Roleta
def roletaViciada(populacao, fitness):
    # Levantamento da qualidade dos indivíduos
    totalFitness = sum(fitness)
    
    # calculando franção de cada individuo na roleta
    fracoes = [f/totalFitness for f in fitness]
    
    # Criando os intervalos da roleta
    intervalos = []
    acum = 0
    for i in fracoes:
        acum+=1
        intervalos.append(round(acum,2))
    
    # Girando a roleta
    resultadoRoleta = random.randint(0, len(populacao))/len(populacao)
    
    # Selecionando um individuo
    ind = 0
    while resultadoRoleta > intervalos[ind]: ind+=1
    
    # Retorna o individuo selecionado
    return populacao[ind]

In [43]:
def roletaViciada(populacao, fitness): # Roleta Honesta
    
    resultado = random.randint(0, len(populacao)-1)
    
    return populacao[resultado]

In [47]:
print(roletaViciada(pop, fit))

[[0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0]]


In [20]:
# Teste Roleta
roletaViciada(pop, fit)

[[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]]

In [82]:
# Função para seleção natural, usando a roleta para escolher os reprodutores
def selecaoNatural(populacao, numCasais):
    selecao = []
    fts = fitness(populacao)
    for i in range(2*numCasais):
        selecao.append(roletaViciada(populacao, fts))
    return selecao

In [83]:
selecaoNatural(pop, 2)

[[[0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 1, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1]],
 [[0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0]]]

In [64]:
# Função para teste de meta
def testeMeta(pops):
    fts = fitness(pops)
    try:
        posMeta = fts.index(1)
        return "Solucao", pops[posMeta]
    except (ValueError):
        return -1

In [80]:
# teste da função de teste de metas
testeMeta(pop)

-1

In [52]:
# Funçao do algoritmo genético de fato
def genetico(maxIteracoes, tamanhoPopulacao, tamGenoma, taxaReproducao, deslocamentoGene):
    
    # Gerar população inicial
    populacao = geraPopulacaoInicial(tamanhoPopulacao, tamGenoma)
    
    # Iterações
    for i in range(maxIteracoes):
        # Primeiro teste de meta
        res = testeMeta(populacao)
        if res != -1: return res,i
        # Etapa de evolucao por reprodução
        novaPopulacao = []
        reprodutores = selecaoNatural(populacao,taxaReproducao)
        machos = reprodutores[:taxaReproducao]
        femeas = reprodutores[taxaReproducao:]
        for k,w in zip(machos, femeas):
            f1,f2 = crossOver(k,w)
            novaPopulacao.append(f1)
            novaPopulacao.append(f2)
        # Teste de meta na nova populacao
        res = testeMeta(novaPopulacao)
        if res!= -1: return res, i
        
        # Aceitar mutações na população
        for z in novaPopulacao:
            if random.randint(0,100)<10:
                z=mutacao(z, deslocamentoGene)
        populacao = novaPopulacao
    return "falha", populacao     
        

In [86]:
# Teste de algoritmo genético
maxIteracoes = 1000
tamanhoPopulacaoInicial = 100
tamGenoma = 8 # tabuleiro n x n
casaisReprodutores = 32 # numero de casais a serem formados para reprodução
deslocamentoGene = 3 # passo no tabuleiro -> regra de mutacao do genoma

genetico(maxIteracoes, tamanhoPopulacaoInicial, tamGenoma, casaisReprodutores, deslocamentoGene)

(('Solucao',
  [[0, 0, 1, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 1, 0, 0, 0, 0],
   [0, 0, 0, 0, 1, 0, 0, 0],
   [0, 0, 0, 0, 0, 1, 0, 0],
   [0, 1, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 1, 0]]),
 98)